# LangChain を使った RAG における埋め込みモデルの比較

LLMを使ったRAGを行う際に埋め込みモデルが必要となりますが、どの程度差がでるのか3種類ほどの埋め込みモデルを使って検証してみたいと思います。

今回試す埋め込みモデル:

* [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large)
* [cl-nagoya/sup-simcse-ja-large](https://huggingface.co/cl-nagoya/sup-simcse-ja-large)
* [pkshatech/GLuCoSE-base-ja](https://huggingface.co/pkshatech/GLuCoSE-base-ja)


参考
こちらを参考にさせていただきました。

* https://github.com/hppRC/simple-simcse-ja

記事内のベンチマークの結果でいうと各モデルはこのような比較となります。

| Model | JSICK (val) | JSICK (test) | JSTS (train) | JSTS (val) | Avg. |
| -- | -- | -- | -- | -- | -- |
| intfloat/multilingual-e5-large | 80.57 | 79.39 | 79.16 | 81.85 | 80.13 |
| cl-nagoya/sup-simcse-ja-large | 84.36 | 83.05 | 79.61 | 83.07 | 81.91 |
| pkshatech/GLuCoSE-base-ja | 76.36 | 75.70 | 78.58 | 81.76 | 78.68 |


前回の記事で試したコードを活用します。LLMには[Elyza 7B Instruct](https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct)を、ChainにはLangchainのQAChainを使ってみました。


## 必要なライブラリをインストール


In [ ]:
# To solve for an error encountered: `NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968`
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 必要なライブラリをインストール
!pip install transformers langchain accelerate bitsandbytes pypdf tiktoken fugashi[unidic-lite] sentence_transformers faiss-gpu trafilatura -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.9 MB/s eta 0:00

## コードを実行

必要なライブラリをロードします。


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate

## データソースを準備

今回はウィキペディア上にある「ONE PIECE」のページをデータソースとして、それに関連する質問をしていきたいと思います。

* https://ja.m.wikipedia.org/wiki/ONE_PIECE

今回はウェブページのテキストだけを抽出してくれる `trafilatura` というライブラリを用いました。

In [ ]:
from trafilatura import fetch_url, extract

url = "https://ja.m.wikipedia.org/wiki/ONE_PIECE"
filename = 'textfile.txt'

document = fetch_url(url)
text = extract(document)
print(text[:1000])

with open(filename, 'w', encoding='utf-8') as f:
    f.write(text)

ONE PIECE
『ONE PIECE』（ワンピース）は、尾田栄一郎による日本の少年漫画作品。『週刊少年ジャンプ』（集英社）にて1997年34号から連載中。略称は「ワンピ」[2]。
|ONE PIECE
|ジャンル
|少年漫画・海賊・冒険
ファンタジー・バトル
|漫画
|作者
|尾田栄一郎
|出版社
|集英社
|
|掲載誌
|週刊少年ジャンプ
|レーベル
|ジャンプ・コミックス
|発表号
|1997年34号 -
|発表期間
|1997年7月22日[1] -
|巻数
|既刊106巻（2023年7月4日現在）
|テンプレート - ノート
|プロジェクト
|漫画
|ポータル
|漫画
概要 編集
海賊王を夢見る少年モンキー・D・ルフィを主人公とする、「ひとつなぎの大秘宝（ワンピース）」を巡る海洋冒険ロマン。
夢への冒険・仲間たちとの友情といったテーマを前面に掲げ、バトルやギャグシーン、感動エピソードをメインとする少年漫画の王道を行く物語として人気を博している[3]。また、長年にわたりながら深く練り込まれた壮大な世界観・巧緻な設定のストーリーも特徴。
2023年7月の時点で単行本は第106巻まで刊行されており、『週刊少年ジャンプ』歴代作品の中では『こちら葛飾区亀有公園前派出所』（1976年 - 2016年）に次ぐ長期連載となっている。国内累計発行部数は2022年時点で日本の漫画では最高となる4億1000万部を突破している[4]。また、第67巻は初版発行部数405万部[5]の国内出版史上最高記録を樹立し、第57巻（2010年3月発売）以降の単行本は初版300万部以上発行を継続する[6]など、出版の国内最高記録をいくつも保持している。
2015年6月15日には "Most Copies Published For The Same Comic Book Series By A Single Author（最も多く発行された単一作者によるコミックシリーズ）" 名義でギネス世界記録に認定された[7][8]。実績は発行部数3億2,086万6,000部（2014年12月時点）[7]。なお、このギネス世界記録は2022年7月付で同作品によって更新され[9]、日本では同年8月に「日本国内累計発行部数4億1656万6000部」と報道された[4]。
本作とともに長年ジャンプの看板作品であった『

抽出したテキストをテキストファイルに保存出来ました。このテキストファイルをLangchainのTextSplitterを使って小口のチャンクに切っていきます。

こうして生成したチャンクからembeddingを生成し、質問のembeddingに一番近いトップｋのチャンクを抽出。そのテキストをプロンプト内に突っ込み、質問と同時にLLMに投げて回答を得る。といった流れとなります。

私は少なくともそういう理解です。

In [ ]:
loader = TextLoader(filename, encoding='utf-8')
documents = loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size=300,
    chunk_overlap=20,
)
texts = text_splitter.split_documents(documents)
print(len(texts))

489


どんな構造をしているのか知るために、何個か見てみましょう。

In [ ]:
texts[30:33]

[Document(page_content='アラバスタ編 編集\n- 【12巻 - 23巻】\n-\n- 偉大なる航路突入（12巻）\n- 麦わらの一味はついに「偉大なる航路」に突入する。リヴァース・マウンテンを降りた場所にある「双子岬」で、仲間の帰還を待ち続けるクジラ・ラブーンと出会う。ルフィはラブーンと、「偉大なる航路」一周後に再戦する約束を交わす。\n- ウイスキーピーク編（12巻 - 13巻）', metadata={'source': 'textfile.txt'}),
 Document(page_content='- ルフィ達は、最初の島「サボテン島」の町「ウイスキーピーク」で大歓迎を受ける。だがその町は、秘密犯罪会社「バロックワークス」（B・W）の社員である賞金稼ぎ達の巣であった。そこで一味は、B・Wエージェントの一人の正体が、「偉大なる航路」にある大国「アラバスタ王国」の王女ネフェルタリ・ビビであると知る。B・Wに潜入していた彼女から、ルフィ達はB・Wによるアラバスタ王国乗っ取り計画を知る。ビビを一行に加えた麦わらの一味は、B・Wからの追手を振り切りつつ、計画を阻止すべく一路アラバスタを目指す。', metadata={'source': 'textfile.txt'}),
 Document(page_content='- リトルガーデン編（13巻 - 15巻）\n- ウイスキーピークを出港したルフィ達は、ジャングルの中で恐竜達が生きる太古の島「リトルガーデン」に上陸する。ルフィ達はその島で、巨人族の二人の戦士・ドリーとブロギーに出会う。彼らは「誇り」を守るため、100年間も決闘を続けてきたという。だがその決闘が、B・Wからの追手による卑劣な策略で邪魔される。ルフィ達はB・Wエージェントにして姑息な美術家・Mr.3らを破り、巨人族の誇りを守る。\n- ドラム島編（15巻 - 17巻）', metadata={'source': 'textfile.txt'})]

## Embeddingの生成とFAISSを使ったベクトルDBの用意

小口のチャンクに切ったテキストを、Embeddingモデルを使ってembeddingに変換していきます。テキストの類似性をもとに検索をできるようにするためです。類似するテキストの抽出数は5に設定しました。

今回３種類の埋め込みモデルを試して最終的な結果を比較していきたいと思います。
下記は `cl-nagoya/sup-simcse-ja-large` を使ったコードとなりますが、それぞれ同じ設定で試しました。

ベクトルDBには `FAISS` のライブラリを用います。（今回はGPUのある環境で走らせてみているため、 faiss-gpu をロードしています。）

In [ ]:
%%time

# embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
embs_model = HuggingFaceEmbeddings(model_name="cl-nagoya/sup-simcse-ja-large")
# embs_model = HuggingFaceEmbeddings(model_name="pkshatech/GLuCoSE-base-ja")

db = FAISS.from_documents(texts, embs_model)

# 一番類似するチャンクをいくつロードするかを変数kに設定出来ます。
retriever = db.as_retriever(search_kwargs={"k": 5})

CPU times: user 22.1 s, sys: 3.86 s, total: 25.9 s
Wall time: 1min 8s


今回の環境ではembeddingを用意するのに19秒程かかりました。前回 `intfloat/multilingual-e5-large` を使った際は25秒ほどだったので少し生成が早いです。

## モデルの用意

LLMにはElyza-7b-instructを用います。

今回もBitsandBytesで4bitに量子化したもので試していきます。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "elyza/ELYZA-japanese-Llama-2-7b-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

次に、Elyza-7b-instruct用のプロンプトテンプレートを用意します。

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "参考情報を元に、ユーザーからの質問にできるだけ正確に答えてください。"
text = "{context}\nユーザからの質問は次のとおりです。{question}"
template = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)

## LLMとChainの指定

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
)
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(
        pipeline=pipe,
        model_kwargs=dict(
            temperature=0.0,
            do_sample=True,
            max_length=512,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.1
        )
    ),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True,
)

## お試し

要約質問ができる状態が整いました。
最初にRAGを使わずにLLMに質問をし、その後にRAGを使って生成してみて差分を比較してみましょう。

In [ ]:
inputs = template.format(context='', question='ニコ・ロビンの職業は何ですか？')
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'[INST] <<SYS>>\n参考情報を元に、ユーザーからの質問にできるだけ正確に答えてください。\n<</SYS>>\n\n\nユーザからの質問は次のとおりです。ニコ・ロビンの職業は何ですか？ [/INST]  ニコ・ロビンの職業は、強盗団の一員です。'

In [ ]:
result = qa("ニコ・ロビンの職業は何ですか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])



> Entering new RetrievalQA chain...

> Finished chain.
回答:  ニコ・ロビンの職業は「海賊」です。

また、他のイベント・企画の情報は以下の通りです。

| イベント・企画
| 詳細
|
| ONE PIECE グランドラインの冒険
| 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。
|
| ONE PIECE お台場の冒険
| 2004年に東京・お台場で開催されたイベント。
|
| ブルック像
| 2003年に熊本県で制作されたブルック像。
|
| フランキー像
| 2003年に熊本県で制作されたフランキー像。
|
| ナミ像
| 2021年7月31日に熊本県で制作されたナミ像。
|
| ロビン像
| 2021年10月9日に熊本県で制作されたロビン像。
|
| ジンベエ像
| 2022年1月22日に熊本県で制作されたジンベエ像。
|
| ブルック像
| 2022年1月22日に熊本県で制作されたブルック像。
|
| フランキー像
| 2022年1月22日に熊本県で制作されたフランキー像。
|
| ナミ像
| 2022年1月22日に熊本県で制作されたナミ像。
|
| ロビン像
| 2022
ソース: [Document(page_content='|ロビン像\n|2021年10月9日\n|南阿蘇村大字河陽 5435番1\n|旧東海大学 阿蘇キャンパス\n|[367]\n|ゾロ像\n|2022年1月22日\n|菊池郡大津町大字大津1156-3\n|大津中央公園\n|[368]\n|ジンベエ像\n|2022年7月23日\n|宇土市住吉町3162-1\n|住吉海岸公園\n|[369]\nその他のイベント・企画 編集\n- ONE PIECE グランドラインの冒険\n- 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。\n- ONE PIECE お台場の冒険', metadata={'source': 'textfile.txt'}), Document(page_content='|2020年11月7日\n|熊本市東区健軍5丁目14-2\n|熊本市動植物園正面入口多目的広場\n|[364]\n|ブルック像\n|2020年11月8日\n|上益城郡御船町辺田見100番地

### ３モデルの比較
* intfloat/multilingual-e5-largeを使った場合
  - 回答:  ニコ・ロビンの職業は「船医」です。
  - ソース: [Document(page_content='空島編 編集\n- 【24巻 - 32巻】\n-\n- ジャヤ編（24巻 - 25巻）\n- アラバスタを後にしたルフィ達は、B・W社副社長であった考古学者ニコ・ロビンを仲間に加える。次の島に向かう航海中、突如空から巨大なガレオン船が落下し、「記録指針（ログポース）」の指す進路が上向きに変更される。それは伝説とされる空に浮かぶ島「空島」への指針を意味していた。', metadata={'source': 'textfile.txt'}), Document(page_content='- 「THE 8TH LOG "SKYPIEA"」2008年4月発行、ISBN 978-4-08-111027-8\n- 「THE 9TH LOG "GOD"」2008年5月発行、ISBN 978-4-08-111028-5\n- 「THE 10TH LOG "BELL"」2008年6月発行、ISBN 978-4-08-111029-2\n- 「THE 11TH LOG "WATER SEVEN"」2009年4月発行、ISBN 978-4-08-111009-4\n- 「THE 12TH LOG "ROCKET MAN"」2009年5月発行、ISBN 978-4-08-111010-0\n- 「THE 13TH LOG "NICO ROBIN"」2009年7月発行、ISBN 978-4-08-111011-7\n- 「THE 14TH LOG "FRANKY"」2009年8月発行、ISBN 978-4-08-111012-4', metadata={'source': 'textfile.txt'}), Document(page_content='- 日清食品「カップヌードル」のテレビCMシリーズ「HUNGRY DAYS」のアニメ作品。第1弾「ゾロ篇」が2019年5月、第2弾「ナミ篇」が2019年9月、第3弾「ビビ篇」が2019年12月、最終回「頂上騎馬戦篇」が2020年2月に公開[424]。「もしONE PIECEのキャラクターが高校生活を送っていたら？」というコンセプトのもと、麦わらの一味の高校生活を、各キャラクターを中心に描く内容となっている。キャラクターデザインは窪之内英策、音楽をBUMP OF CHICKEN、ナレーションを林原めぐみが手がけ、集英社と元々のアニメシリーズを制作している東映アニメーションの監修の元、シャフトがアニメーション制作を担当。声優陣はルフィ役の田中真弓やゾロ役の中井和哉といったアニメシリーズのキャストが出演している[425]。', metadata={'source': 'textfile.txt'}), Document(page_content='- サンジ\n- 声 - 平田広明\n- 麦わらの一味コック。コックの命である手を傷つけないように、足技で戦う。無類の女好き。伝説の海「オールブルー」を探す。\n- トニートニー・チョッパー\n- 声 - 大谷育江\n- 麦わらの一味船医。「ヒトヒトの実」を食べ人の能力を持った人間トナカイ。万能薬（何でも治せる医者）を目指している。\n- ニコ・ロビン\n- 声 - 山口由里子', metadata={'source': 'textfile.txt'}), Document(page_content='|ロビン像\n|2021年10月9日\n|南阿蘇村大字河陽 5435番1\n|旧東海大学 阿蘇キャンパス\n|[367]\n|ゾロ像\n|2022年1月22日\n|菊池郡大津町大字大津1156-3\n|大津中央公園\n|[368]\n|ジンベエ像\n|2022年7月23日\n|宇土市住吉町3162-1\n|住吉海岸公園\n|[369]\nその他のイベント・企画 編集\n- ONE PIECE グランドラインの冒険\n- 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。\n- ONE PIECE お台場の冒険', metadata={'source': 'textfile.txt'})]
  - 考察: 必要な文章は抽出できましたが、４つ目に抽出されたテキストが主にチョッパーに関する文章だったところ、一番最後に「ニコ・ロビン」が出てきていることから、ロビンの職業を「船医」と回答してしまったと考えられます。

* cl-nagoya/sup-simcse-ja-largeを使った場合
  - 回答:  ニコ・ロビンの職業は「海賊」です。
    また、他のイベント・企画の情報は以下の通りです。

    | イベント・企画
    | 詳細
    |
    | ONE PIECE グランドラインの冒険
    | 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。
    |
    | ONE PIECE お台場の冒険
    | 2004年に東京・お台場で開催されたイベント。
    |
    | ブルック像
    | 2003年に熊本県で制作されたブルック像。
    |
    | フランキー像
    | 2003年に熊本県で制作されたフランキー像。
    |
    | ナミ像
    | 2021年7月31日に熊本県で制作されたナミ像。
    |
    | ロビン像
    | 2021年10月9日に熊本県で制作されたロビン像。
    |
    | ジンベエ像
    | 2022年1月22日に熊本県で制作されたジンベエ像。
    |
    | ブルック像
    | 2022年1月22日に熊本県で制作されたブルック像。
    |
    | フランキー像
    | 2022年1月22日に熊本県で制作されたフランキー像。
    |
    | ナミ像
    | 2022年1月22日に熊本県で制作されたナミ像。
    |
    | ロビン像
    | 2022
  - ソース: [Document(page_content='|ロビン像\n|2021年10月9日\n|南阿蘇村大字河陽 5435番1\n|旧東海大学 阿蘇キャンパス\n|[367]\n|ゾロ像\n|2022年1月22日\n|菊池郡大津町大字大津1156-3\n|大津中央公園\n|[368]\n|ジンベエ像\n|2022年7月23日\n|宇土市住吉町3162-1\n|住吉海岸公園\n|[369]\nその他のイベント・企画 編集\n- ONE PIECE グランドラインの冒険\n- 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。\n- ONE PIECE お台場の冒険', metadata={'source': 'textfile.txt'}), Document(page_content='|2020年11月7日\n|熊本市東区健軍5丁目14-2\n|熊本市動植物園正面入口多目的広場\n|[364]\n|ブルック像\n|2020年11月8日\n|上益城郡御船町辺田見100番地\n|ふれあい広場\n|[364]\n|フランキー像\n|2020年11月21日\n|阿蘇郡高森町1526-5\n|高森駅前\n|[365]\n|ナミ像\n|2021年7月31日\n|阿蘇郡西原村小森 2115-3\n|俵山交流館 萌の里\n|[366]\n|ロビン像\n|2021年10月9日', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ アニメでは第46話・第47話（2000年11月1日・11月8日放送）。\n- ^ アニメでは第68話・第69話（2001年5月13日・5月20日放送）。\n- ^ 映画『ジャンゴのダンスカーニバル』（同時上映の短編）の原案としてアニメ化。\n- ^ アニメでは第778話（2017年2月26日放送）で、ワポル本人の回想という形でアニメ化。\n- ^ アニメでは第510話・第514話（2011年8月14日・9月11日放送）。', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ "AMDアワード、優秀賞に「ウタ」「ELDEN RING」「きつねダンス」など10件". INTERNET Watch. 2023年2月14日. 2023年3月21日閲覧。\n- ^ "H.I.S.プレゼンツ ONE PIECE メモリアルログ in ハウステンボス". ハウステンボス. 2010年5月24日. 2011年1月10日閲覧。\n- ^ "ハウステンボス、1億7000万円の黒字". YOMIURI ONLINE. 読売新聞社. 2010年9月2日. 2010年9月6日時点のオリジナルよりアーカイブ。2011年1月10日閲覧。\n- ^ ソウルのワンピース展、一転開催へ 26日から 朝日新聞、2014年7月25日', metadata={'source': 'textfile.txt'}), Document(page_content='- 〜あらゆる世界を見学せよ〜潜入!リアルスコープ\n- 2010年6月26日、2011年1月8日、2012年8月25日にアニメの制作現場が、2011年3月19日にはジャンプの製作現場が特集された。\n- めざましテレビ\n- 2014年8月25日から30日の6日間、番組内の名物コーナー「めざましじゃんけん」に日替わりでルフィ・ゾロ・ナミ・ウソップ・サンジ・チョッパーが登場した[410]。\n- 世にも奇妙な物語', metadata={'source': 'textfile.txt'})]
  - 考察: ベクトル検索時に必要な文章を抽出出来ませんでした。

* pkshatech/GLuCoSE-base-japを使った場合
  - 回答:  ニコ・ロビンの職業は「海賊」です。
  - ソース: [Document(page_content='- ロビンの真意は、「世界政府の強大すぎる力が仲間に向けられるのが怖い」というものであった。彼女の想いを聞いたルフィは世界政府に宣戦布告し、一味はCP9との全面対決に挑む。CP9最強の男ロブ・ルッチらとの激戦の末、ロビン奪還に成功した一味は、「バスターコール」による海軍の猛攻が迫る窮地の最中、戦場に現れたメリー号の助けもあり、エニエス・ロビーからの脱出に成功する。しかし、ついにメリー号が限界に達してしまう。ルフィ達はメリー号を火葬し、東の海から航海を共にしたメリー号に別れを告げた。', metadata={'source': 'textfile.txt'}), Document(page_content='- ルフィ達はロビンを奪還するため、ガレーラカンパニー社員・フランキー一家らと協力し、司法の島「エニエス・ロビー」に乗り込む。その中には、仮面を被り謎のヒーロー「そげキング」と名乗るウソップの姿もあった。\n- エニエス・ロビー編（39巻 - 46巻）', metadata={'source': 'textfile.txt'}), Document(page_content='イタリア・セリエA所属のバレーボールクラブ『パッラヴォーロ・モデナ』のリベロ用のユニフォームのデザインに採用された[253]。\n北アメリカ 編集\nカナダの2010年オタワ国際アニメーションフェスティバルにおいて長編作品部門に日本から『ONE PIECE FILM STRONG WORLD』と『緑子』の2作品がノミネートされた。『ONE PIECE FILM STRONG WORLD』は日本のエンタテイメント作品としては異例となる連続出品を果たした[254]。また2010年第12回スペイン・バルセロナアジア映画祭において観客賞（オーディエンスアワード）を受賞した。', metadata={'source': 'textfile.txt'}), Document(page_content='- 船を求め立ち寄った「シロップ村」で、ルフィはウソつきの少年ウソップと出会う。ウソップは、シャンクスの部下ヤソップの息子であった。ルフィ達は村の富豪の娘カヤの屋敷を訪れるが、執事クラハドールに追い返される。しかしルフィとウソップはしばらくして、クラハドールがかつて処刑されたはずの海賊〝キャプテン・クロ〟であることを知ってしまう。クロは自分の海賊団に村を襲わせ、カヤを殺して財産を手に入れようと企んでいた。ルフィ達はウソップと共にクロネコ海賊団を迎え撃ち、クロの計画を阻止する。ルフィ達は、新たに狙撃手ウソップを仲間に加え、さらにカヤから海賊船「ゴーイングメリー号」を譲り受ける。', metadata={'source': 'textfile.txt'}), Document(page_content='登場人物 編集\nここでは、主人公とその仲間「麦わらの一味」のみを紹介する。\n- モンキー・D・ルフィ\n- 声 - 田中真弓\n- 本作の主人公。麦わらの一味船長。「ゴムゴムの実」の能力者のゴム人間。麦わら帽子がトレードマーク。夢は「ひとつなぎの大秘宝（ワンピース）」を見つけて「海賊王」になること。\n- ロロノア・ゾロ\n- 声 - 中井和哉', metadata={'source': 'textfile.txt'})]
  - 考察: ベクトル検索時に必要な文章を抽出出来ませんでしたが、回答は正解と言えるかと思います。

In [ ]:
inputs = template.format(context='', question='エネルは何者ですか？')
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

'[INST] <<SYS>>\n参考情報を元に、ユーザーからの質問にできるだけ正確に答えてください。\n<</SYS>>\n\n\nユーザからの質問は次のとおりです。エネルは何者ですか？ [/INST]  エネルは、エネルギーのことです。'

In [ ]:
result = qa("エネルは何者ですか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])



> Entering new RetrievalQA chain...

> Finished chain.
回答:  エネルは、ONE PIECEの世界に登場する架空の人物です。

エネルは、東海大学の教授であり、ルフィの師匠でもあります。彼はルフィに海賊の技術を教えています。また、エネルはサンジの父親でもあります。
ソース: [Document(page_content='|ロビン像\n|2021年10月9日\n|南阿蘇村大字河陽 5435番1\n|旧東海大学 阿蘇キャンパス\n|[367]\n|ゾロ像\n|2022年1月22日\n|菊池郡大津町大字大津1156-3\n|大津中央公園\n|[368]\n|ジンベエ像\n|2022年7月23日\n|宇土市住吉町3162-1\n|住吉海岸公園\n|[369]\nその他のイベント・企画 編集\n- ONE PIECE グランドラインの冒険\n- 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。\n- ONE PIECE お台場の冒険', metadata={'source': 'textfile.txt'}), Document(page_content='|2010年12月23日 - 2011年1月10日\n|九族文化村\n|台湾・南投県魚池郷\n|2011年7月 - 2012年4月\n2012年8月 - 2013年6月\n|秋田拠点センターアルヴェ\n|秋田県秋田市\n|2011年8月10日 - 18日\n|横浜ランドマークタワー\n|神奈川県横浜市\n|2011年11月10日 - 12月25日\n|NTTクレドホール\n|広島県広島市\n|2011年12月23日 - 2012年1月9日\n|ひらかたパーク\n|大阪府枚方市\n|2012年3月7日 - 7月8日\n|ルスツリゾート\n|北海道虻田郡', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ フジテレビ制作の番組がアジアン・テレビジョン・アワードで最優秀賞受賞!!『わ・す・れ・な・い 東日本大震災155日の記録』 Archived 2014-04-03 at the Wayback Machine. フジテレビ、2012年12月10日\n- ^ "

### ３モデルの比較
* intfloat/multilingual-e5-largeを使った場合
  - 回答:  エネルは神の国「スカイピア」の神の一人である「神・エネル」です。
  - ソース: [Document(page_content='- ルフィ達は上空1万メートルにある空島に辿り着く。そこには今まで全く見たことがない未知の文化が広がっていた。ルフィ達は、神の国「スカイピア」で上陸した「神の島（アッパーヤード）」が、かつて地上に存在した伝説の黄金郷であることをつきとめる。しかし、そこは神の軍団を率いる〝神・エネル〟が支配する土地であり、空の民と島の先住民シャンディアが400年に渡り争い続けている土地であった。黄金捜しに乗り出したルフィ達は、神の軍団とシャンディアとの過酷なサバイバルに巻き込まれる。エネルの圧倒的な力に多くの戦士たちが倒れていき、エネルによって空島は消滅の危機に陥る。だが、唯一エネルに対抗できるルフィによって空島の危機は防がれ、400年に渡る空の民とシャンディアの争いに終止符が打たれた。', metadata={'source': 'textfile.txt'}), Document(page_content='- 一方、サニー号は巨大なロボットに捕まってエッグヘッドに連行される。ゾロたちはベガパンクの分身の「悪(リリス)」と「正(シャカ)」により、研究所に通されることになる。研究所に着くと、一味はジンベエそっくりの新型パシフィスタ「セラフィム」の襲撃を受け戦闘データが収集されるが、セラフィムを破壊される前に正(シャカ)が戦闘を中止させる。正(シャカ)は、この島が「過去」であり、この島のような高度な文明を持った王国が900年前に実在していたと語る。', metadata={'source': 'textfile.txt'}), Document(page_content='ノベライズ作品 編集\n集英社の新書レーベル「JUMP j BOOKS」より発売されている、アニメオリジナルストーリーや劇場版のノベライズ作品。一部は児童文学レーベル「集英社みらい文庫」でも刊行されている。\nその他の小説作品 編集\n- ONE PIECE novel A（エース）\n- エースを主人公とし、スペード海賊団時代の冒険を描く。ムック『ONE PIECE magazine』Vol.1からVol.3まで連載され[160][161]、後に第1巻として書籍化された。著者はひなたしょう。', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ "ワンピースが新聞に！週刊で4週連続発行". 日刊スポーツ. 2012年11月13日. 2012年11月25日閲覧。\n- ^ "『ONE PIECE』とコラボで実写CM ルフィに斎藤工、ゾロに池内博之、サンジに窪塚洋介 ナミを泉里香". オリコンニュース. オリコン. 2018年12月31日. 2019年1月1日閲覧。\n- ^ "夏木マリ、『ONE PIECE』Dr.くれは完全再現 139歳役に嘆きも「ハッピー? ガキ共!!」". オリコンニュース. オリコン. 2019年2月14日. 2019年2月22日閲覧。', metadata={'source': 'textfile.txt'}), Document(page_content='|2003年5月\n|8200万部[194][信頼性要検証]\n|2005年2月\n|1億部[17][12]\n|2007年7月\n|1億4000万部[195]\n|2009年10月\n|1億7000万部[196]\n|2010年11月\n|2億部[17][197]\n|2011年5月\n|2億3000万部[198]\n|2011年8月\n|2億4000万部[199]\n|2011年11月\n|2億5000万部[179]\n|2012年2月\n|2億6000万部[180]\n|2012年11月\n|2億8000万部[182]\n|2013年6月\n|2億9000万部[184]\n|2013年11月\n|3億部[17]\n|2014年5月\n|3億1000万部[187]', metadata={'source': 'textfile.txt'})]
  - 考察: 最適な文章を抽出できました。回答も正解と言えるかと思います。

* cl-nagoya/sup-simcse-ja-largeを使った場合
  - 回答:  エネルは、ONE PIECEの世界に登場する架空の人物です。

    エネルは、東海大学の教授であり、ルフィの師匠でもあります。彼はルフィに海賊の技術を教えています。また、エネルはサンジの父親でもあります。
  - ソース: [Document(page_content='|ロビン像\n|2021年10月9日\n|南阿蘇村大字河陽 5435番1\n|旧東海大学 阿蘇キャンパス\n|[367]\n|ゾロ像\n|2022年1月22日\n|菊池郡大津町大字大津1156-3\n|大津中央公園\n|[368]\n|ジンベエ像\n|2022年7月23日\n|宇土市住吉町3162-1\n|住吉海岸公園\n|[369]\nその他のイベント・企画 編集\n- ONE PIECE グランドラインの冒険\n- 2002年に東京・サンリオピューロランドで上映された初のフルCGアニメ。\n- ONE PIECE お台場の冒険', metadata={'source': 'textfile.txt'}), Document(page_content='|2020年11月7日\n|熊本市東区健軍5丁目14-2\n|熊本市動植物園正面入口多目的広場\n|[364]\n|ブルック像\n|2020年11月8日\n|上益城郡御船町辺田見100番地\n|ふれあい広場\n|[364]\n|フランキー像\n|2020年11月21日\n|阿蘇郡高森町1526-5\n|高森駅前\n|[365]\n|ナミ像\n|2021年7月31日\n|阿蘇郡西原村小森 2115-3\n|俵山交流館 萌の里\n|[366]\n|ロビン像\n|2021年10月9日', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ アニメでは第46話・第47話（2000年11月1日・11月8日放送）。\n- ^ アニメでは第68話・第69話（2001年5月13日・5月20日放送）。\n- ^ 映画『ジャンゴのダンスカーニバル』（同時上映の短編）の原案としてアニメ化。\n- ^ アニメでは第778話（2017年2月26日放送）で、ワポル本人の回想という形でアニメ化。\n- ^ アニメでは第510話・第514話（2011年8月14日・9月11日放送）。', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ "AMDアワード、優秀賞に「ウタ」「ELDEN RING」「きつねダンス」など10件". INTERNET Watch. 2023年2月14日. 2023年3月21日閲覧。\n- ^ "H.I.S.プレゼンツ ONE PIECE メモリアルログ in ハウステンボス". ハウステンボス. 2010年5月24日. 2011年1月10日閲覧。\n- ^ "ハウステンボス、1億7000万円の黒字". YOMIURI ONLINE. 読売新聞社. 2010年9月2日. 2010年9月6日時点のオリジナルよりアーカイブ。2011年1月10日閲覧。\n- ^ ソウルのワンピース展、一転開催へ 26日から 朝日新聞、2014年7月25日', metadata={'source': 'textfile.txt'}), Document(page_content='- 〜あらゆる世界を見学せよ〜潜入!リアルスコープ\n- 2010年6月26日、2011年1月8日、2012年8月25日にアニメの制作現場が、2011年3月19日にはジャンプの製作現場が特集された。\n- めざましテレビ\n- 2014年8月25日から30日の6日間、番組内の名物コーナー「めざましじゃんけん」に日替わりでルフィ・ゾロ・ナミ・ウソップ・サンジ・チョッパーが登場した[410]。\n- 世にも奇妙な物語', metadata={'source': 'textfile.txt'})]
  - 考察: ベクトル検索時に必要な文章を抽出出来ませんでした。ハルシネーションとなってしまいました。

* pkshatech/GLuCoSE-base-japを使った場合
  - 回答:  エネルは、ワノ国を出港した麦わらの一味が上空1万メートルにある空島に辿り着く「神の国「スカイピア」」の神の軍団を率いる「神・エネル〞です。
  - ソース: [Document(page_content='- ワノ国を出港した麦わらの一味は、とある海域で巨大な暖水渦に遭遇し、渦に巻き込まれていた女を助ける。その女は、最悪の世代の一人のジュエリー・ボニーであった。しかしその直後、サニー号が武装した巨大な鮫に襲われ、一味は離れ離れになってとある島に漂着。ボニーによると、そこはDr.ベガパンクが研究拠点にしている政府の島「エッグヘッド」だという。ルフィ達はボニーと共に島を散策するが、そこは途轍もなく高度な技術で作られた未来都市そのものであった。散策の最中、ルフィ達はパシフィスタに遭遇し、戦おうとするがボニーに妨害される。ボニーは、バーソロミュー・くまは自分の父親で、ベガパンクによって改造人間にされ自我を失ったと語る。彼女は、ベガパンクに父を改造人間にした理由を問いただすためにエッグヘッドを訪れたのであった。', metadata={'source': 'textfile.txt'}), Document(page_content='- ロビンの真意は、「世界政府の強大すぎる力が仲間に向けられるのが怖い」というものであった。彼女の想いを聞いたルフィは世界政府に宣戦布告し、一味はCP9との全面対決に挑む。CP9最強の男ロブ・ルッチらとの激戦の末、ロビン奪還に成功した一味は、「バスターコール」による海軍の猛攻が迫る窮地の最中、戦場に現れたメリー号の助けもあり、エニエス・ロビーからの脱出に成功する。しかし、ついにメリー号が限界に達してしまう。ルフィ達はメリー号を火葬し、東の海から航海を共にしたメリー号に別れを告げた。', metadata={'source': 'textfile.txt'}), Document(page_content='- 深海1万メートルにある海底の楽園「魚人島」に到着した麦わらの一味は、島を治める「リュウグウ王国」の国王ネプチューンの招待を受ける。王宮でルフィが出会った王女・しらほし姫は、彼女を付け狙う海賊バンダー・デッケン九世から身を守るため、軟禁状態に置かれていた。一方、ホーディ・ジョーンズ率いる「新魚人海賊団」が、デッケンの一味と手を組み、リュウグウ王国崩壊を目論み動き出す。新魚人海賊団は、かつてルフィが倒した魚人の海賊アーロンの意志を継ぎ、人間たちへの復讐を企てていた。ホーディは、魚人島の新たな王となり、人間との友好を望む者を排除するため王国の乗っ取りを宣言する。', metadata={'source': 'textfile.txt'}), Document(page_content='- 女ヶ島編（53巻 - 54巻）\n- ルフィが飛ばされたのは、「凪の帯」にある女ヶ島「アマゾン・リリー」。そこは、七武海の紅一点ボア・ハンコックが治める、男子禁制の女人国であった。一時は処刑されそうになるが、彼の恐れを知らぬ度胸と器量の大きさに惚れ込み、ルフィはハンコックに恋心を抱かれるようになる。', metadata={'source': 'textfile.txt'}), Document(page_content='- ルフィ達は上空1万メートルにある空島に辿り着く。そこには今まで全く見たことがない未知の文化が広がっていた。ルフィ達は、神の国「スカイピア」で上陸した「神の島（アッパーヤード）」が、かつて地上に存在した伝説の黄金郷であることをつきとめる。しかし、そこは神の軍団を率いる〝神・エネル〟が支配する土地であり、空の民と島の先住民シャンディアが400年に渡り争い続けている土地であった。黄金捜しに乗り出したルフィ達は、神の軍団とシャンディアとの過酷なサバイバルに巻き込まれる。エネルの圧倒的な力に多くの戦士たちが倒れていき、エネルによって空島は消滅の危機に陥る。だが、唯一エネルに対抗できるルフィによって空島の危機は防がれ、400年に渡る空の民とシャンディアの争いに終止符が打たれた。', metadata={'source': 'textfile.txt'})]
  - 考察: ベクトル検索時に5つ目の結果として関連した文章を抽出できましたが、他の結果が関連性の低い文章となってしまい、最終的な回答がズレました。

In [ ]:
inputs = template.format(context='', question='チョッパーの特殊能力は何ですか？')
inputs = tokenizer(inputs, return_tensors='pt').to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
output

'[INST] <<SYS>>\n参考情報を元に、ユーザーからの質問にできるだけ正確に答えてください。\n<</SYS>>\n\n\nユーザからの質問は次のとおりです。チョッパーの特殊能力は何ですか？ [/INST]  チョッパーの特殊能力について回答いたします。\n\nチョッパーは、相手の攻撃を受けてもその攻撃を相手に返してくることのできる「リフレクター」という能力を持っています。これにより、チョッパーは攻撃を受けることが多く、守備力が低いという傾向にあります。'

In [ ]:
result = qa("チョッパーの特殊能力は何ですか？")
print('回答:', result['result'])
print('='*10)
print('ソース:', result['source_documents'])



> Entering new RetrievalQA chain...

> Finished chain.
回答:  チョッパーの特殊能力について回答いたします。

チョッパーの特殊能力は、「足技で戦う」ことができることです。彼は無類の女好きであり、伝説の海「オールブルー」を探す一味の仲間です。
ソース: [Document(page_content='- ルフィ達はロビンを奪還するため、ガレーラカンパニー社員・フランキー一家らと協力し、司法の島「エニエス・ロビー」に乗り込む。その中には、仮面を被り謎のヒーロー「そげキング」と名乗るウソップの姿もあった。\n- エニエス・ロビー編（39巻 - 46巻）', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ "「ワンピース・プレミアショー 2014」第2弾 本日、満員御礼でスタート!!". 財経新聞. 2014年7月4日. 2018年11月14日閲覧。\n- ^ a b "仏のマンガ・アニメグランプリ 「FAIRY TAIL」「べるぜバブ」等が受賞". アニメ!アニメ!. イード. 2012年4月9日. 2022年9月10日閲覧。\n- ^ "ワンピ×パリ水族館企画レポ、館内に存在感抜群の巨大チョッパーも。". ナリナリドットコム. 2013年5月2日. 2013年7月6日閲覧。', metadata={'source': 'textfile.txt'}), Document(page_content='- 11月11日 - 単行本国内累計発行部数が2億冊を突破（第60巻）[14]。\n- 2011年（平成23年）\n- 4月 - 『週刊少年ジャンプ』2011年17号に島袋光年の『トリコ』とのクロスオーバー作品『実食! 悪魔の実!!』が掲載される。\n- 12月3日 - 『最強ジャンプ』2012年1月号より、スピンオフ漫画『チョッパーマン』が連載開始。\n- 2012年（平成24年）', metadata={'source': 'textfile.txt'}), Document(page_content='- サンジ\n- 声 - 平田広明\n- 麦わらの一味コック。コックの命である手を傷つけないように、足技で戦う。無類の女好き。伝

### ３モデルの比較
* intfloat/multilingual-e5-largeを使った場合
  - 回答:  チョッパーの特殊能力は「トナミの術」です。

    - トナミの術
    - 敵を睡眠状態にすることができる。
    - ワンピースパーティー
    - ワンピースパーティーでは、チョッパーが「トナミの術」を使って敵を睡眠状態にする描写があります。

    また、「ワンピース」のスピンオフ作品として、「チョッパーマン」が2012年1月号から2014年2月号まで連載されていました。
  - ソース: [Document(page_content='- チョッパーマン\n- パラレルワールドを舞台に、チョッパーを主人公にしたスピンオフ漫画。作画は武井宏文。『最強ジャンプ』2012年1月号から2014年2月号まで連載された。\n- ワンピースパーティー\n- SD化したキャラたちが繰り広げる、スピンオフギャグ漫画。作画は安藤英。『最強ジャンプ』2015年1月号より連載中。\n- CHIN PIECE[36]', metadata={'source': 'textfile.txt'}), Document(page_content='- 11月11日 - 単行本国内累計発行部数が2億冊を突破（第60巻）[14]。\n- 2011年（平成23年）\n- 4月 - 『週刊少年ジャンプ』2011年17号に島袋光年の『トリコ』とのクロスオーバー作品『実食! 悪魔の実!!』が掲載される。\n- 12月3日 - 『最強ジャンプ』2012年1月号より、スピンオフ漫画『チョッパーマン』が連載開始。\n- 2012年（平成24年）', metadata={'source': 'textfile.txt'}), Document(page_content='- リトルガーデン出港後、ナミが急病に倒れてしまう。急遽進路を変更し、雪の島「ドラム島」に立ち寄った麦わらの一味は、悪魔の実を食べ人の能力を持ったトナカイ、トニートニー・チョッパーと出会う。ルフィはチョッパーを仲間に誘うが、彼には悲しき過去があった。そこへ、かつて島で悪政を敷いた元ドラム王国国王ワポルが帰還する。ルフィ達はチョッパーと共闘してワポルを撃退し、船医チョッパーを仲間に迎える。\n- アラバスタ編（17巻 - 23巻）', metadata={'source': 'textfile.txt'}), Document(page_content='- オリコンの「最も友達になりたいテレビアニメのキャラは?」では1位の「ドラえもん」に次ぎ2位に「モンキー・D・ルフィ」、5位に「トニートニー・チョッパー」がランクインし、同じくオリコンの「漫画の"名脇役"ランキング」では2位に「ロロノア・ゾロ」、5位に「トニートニー・チョッパー」がランクインした。どちらもトップ5に同作から2キャラクターランクインしたのは『ONE PIECE』のみとなっている。\n- TSUTAYA onlineが行った「子供に観せたいアニメ映画ランキング」で『ONE PIECE』の劇場版シリーズが7位にランクインした[242]。', metadata={'source': 'textfile.txt'}), Document(page_content='- 笑う犬の発見\n- 番組冒頭に、「犬!PIECE」というパロディコーナーが設けられ、主人公のルフィもアニメのまま登場した（コーナー内では原田泰造がウソップ、名倉潤が『ピピプー』という番組オリジナルキャラクター、チョッパーが着ぐるみで登場した）。\n- FNS地球特捜隊ダイバスター\n- 特番「ダイバスター特別編 ONE PIECEの裏側 本当にちょっとだけ見せますスペシャル」（2006年9月25日放送）で本作のスタッフ、アフレコスタジオなどが登場した。', metadata={'source': 'textfile.txt'})]
  - 考察: ベクトル検索時に必要な文章を抽出出来ませんでした。ハルシネーションとなってしまいました。

* cl-nagoya/sup-simcse-ja-largeを使った場合
  - 回答:  チョッパーの特殊能力について回答いたします。

    チョッパーの特殊能力は、「足技で戦う」ことができることです。彼は無類の女好きであり、伝説の海「オールブルー」を探す一味の仲間です。
  - ソース: [Document(page_content='- ルフィ達はロビンを奪還するため、ガレーラカンパニー社員・フランキー一家らと協力し、司法の島「エニエス・ロビー」に乗り込む。その中には、仮面を被り謎のヒーロー「そげキング」と名乗るウソップの姿もあった。\n- エニエス・ロビー編（39巻 - 46巻）', metadata={'source': 'textfile.txt'}), Document(page_content='- ^ "「ワンピース・プレミアショー 2014」第2弾 本日、満員御礼でスタート!!". 財経新聞. 2014年7月4日. 2018年11月14日閲覧。\n- ^ a b "仏のマンガ・アニメグランプリ 「FAIRY TAIL」「べるぜバブ」等が受賞". アニメ!アニメ!. イード. 2012年4月9日. 2022年9月10日閲覧。\n- ^ "ワンピ×パリ水族館企画レポ、館内に存在感抜群の巨大チョッパーも。". ナリナリドットコム. 2013年5月2日. 2013年7月6日閲覧。', metadata={'source': 'textfile.txt'}), Document(page_content='- 11月11日 - 単行本国内累計発行部数が2億冊を突破（第60巻）[14]。\n- 2011年（平成23年）\n- 4月 - 『週刊少年ジャンプ』2011年17号に島袋光年の『トリコ』とのクロスオーバー作品『実食! 悪魔の実!!』が掲載される。\n- 12月3日 - 『最強ジャンプ』2012年1月号より、スピンオフ漫画『チョッパーマン』が連載開始。\n- 2012年（平成24年）', metadata={'source': 'textfile.txt'}), Document(page_content='- サンジ\n- 声 - 平田広明\n- 麦わらの一味コック。コックの命である手を傷つけないように、足技で戦う。無類の女好き。伝説の海「オールブルー」を探す。\n- トニートニー・チョッパー\n- 声 - 大谷育江\n- 麦わらの一味船医。「ヒトヒトの実」を食べ人の能力を持った人間トナカイ。万能薬（何でも治せる医者）を目指している。\n- ニコ・ロビン\n- 声 - 山口由里子', metadata={'source': 'textfile.txt'}), Document(page_content='ウォーターセブン編 編集\n- 【32巻 - 46巻】\n-\n- ロングリングロングランド編（32巻 - 34巻）\n- 空島から帰還したルフィ達は、ロングリングロングランドで、フォクシー海賊団から仲間をかけた海賊のゲーム「デービーバックファイト」を挑まれる。度重なる妨害に遭い、一時チョッパーを奪われるも、フォクシーとの直接対決を制し勝利を収めた。その直後、海軍本部大将〝青雉〟が現れ、世界政府最高戦力の実力を見せつけられた。', metadata={'source': 'textfile.txt'})]
  - 考察: ベクトル検索時に4つ目の結果として必要な文章を抽出できましたが、サンジの説明文の最後に「チョッパー」のテキストが入ってしまったことで誤った回答となってしまいました。RAGのセットアップの問題ですね。

* pkshatech/GLuCoSE-base-japを使った場合
  - 回答:  チョッパーの特殊能力は、「ヒトヒトの実」を食べたことにより、人の能力を持った人間トナカイである「トナカイ人間」の能力です。彼の場合は、「万能薬（何でも治す医者）」を目指しています。
  - ソース: [Document(page_content='- サンジ\n- 声 - 平田広明\n- 麦わらの一味コック。コックの命である手を傷つけないように、足技で戦う。無類の女好き。伝説の海「オールブルー」を探す。\n- トニートニー・チョッパー\n- 声 - 大谷育江\n- 麦わらの一味船医。「ヒトヒトの実」を食べ人の能力を持った人間トナカイ。万能薬（何でも治せる医者）を目指している。\n- ニコ・ロビン\n- 声 - 山口由里子', metadata={'source': 'textfile.txt'}), Document(page_content='- 声 - 山口由里子\n- 麦わらの一味考古学者。「ハナハナの実」の能力者。歴史上の「空白の100年」の謎を解き明かすため旅をしている。\n- フランキー\n- 声 - 矢尾一樹\n- 麦わらの一味船大工。体中に武器を仕込んだサイボーグ。自分の作った船に乗り、その船が海の果てに辿り着くのを見届けることが夢。\n- ブルック\n- 声 - チョー', metadata={'source': 'textfile.txt'}), Document(page_content='- 11月11日 - 単行本国内累計発行部数が2億冊を突破（第60巻）[14]。\n- 2011年（平成23年）\n- 4月 - 『週刊少年ジャンプ』2011年17号に島袋光年の『トリコ』とのクロスオーバー作品『実食! 悪魔の実!!』が掲載される。\n- 12月3日 - 『最強ジャンプ』2012年1月号より、スピンオフ漫画『チョッパーマン』が連載開始。\n- 2012年（平成24年）', metadata={'source': 'textfile.txt'}), Document(page_content='- チョッパーマン\n- パラレルワールドを舞台に、チョッパーを主人公にしたスピンオフ漫画。作画は武井宏文。『最強ジャンプ』2012年1月号から2014年2月号まで連載された。\n- ワンピースパーティー\n- SD化したキャラたちが繰り広げる、スピンオフギャグ漫画。作画は安藤英。『最強ジャンプ』2015年1月号より連載中。\n- CHIN PIECE[36]', metadata={'source': 'textfile.txt'}), Document(page_content='- 笑う犬の発見\n- 番組冒頭に、「犬!PIECE」というパロディコーナーが設けられ、主人公のルフィもアニメのまま登場した（コーナー内では原田泰造がウソップ、名倉潤が『ピピプー』という番組オリジナルキャラクター、チョッパーが着ぐるみで登場した）。\n- FNS地球特捜隊ダイバスター\n- 特番「ダイバスター特別編 ONE PIECEの裏側 本当にちょっとだけ見せますスペシャル」（2006年9月25日放送）で本作のスタッフ、アフレコスタジオなどが登場した。', metadata={'source': 'textfile.txt'})]
  - 考察: 正解が出ました。ベクトル検索時に一番関連性の高い文章が抽出できました。



## 結論

今回Wikipedia記事をLangChainのCharacterTextSplitterを使って、３種類の埋め込みモデルを使ってベクトル化し、RAGによる質問応答を試しました。

* 今回たった３つの質問ではありましたが、埋め込みモデルによって抽出される文章は結構変わってくることがわかりました。
* 主観ですが、今回のRAGにおけるベクトル検索のクオリティは、`intfloat/multilingual-e5-large` >= `pkshatech/GLuCoSE-base-jap` > `cl-nagoya/sup-simcse-ja-large` というような感触でした。使い方が間違っていたり、適したフォーマットになっていない等あるのかもしれません。
* RAG用のセットアップのチューニングがかなり重要だというのは、今回の実験で改めて感じました。埋め込みモデルを変える度にチューニングが必要になるのでなかなか埋め込みモデルを変えられなそうです。変える機会がたくさんあるわけではないですが…。
* 結局、CharacterTextSplitter以外の、より高度な手法を用いたRAGを行ったほうが良さそうです。

今回使ったColab:
* `intfloat/multilingual-e5-large`: https://colab.research.google.com/drive/1Fe3LEtkTfLHpGM94gfznYoIhabm_QLY8?usp=sharing
* `cl-nagoya/sup-simcse-ja-large`: https://colab.research.google.com/drive/1Bjw42JSfwwJz19txvFfxWweqchFUkdD2?usp=sharing
* `pkshatech/GLuCoSE-base-jap`: https://colab.research.google.com/drive/1Q_hx3yl0rs3MlW4jeJ4Mwh7HLEqEUs0Y?usp=sharing
* `openai/text-embedding-ada-002`: https://colab.research.google.com/drive/1ytUMoUOWlR_83ti6jRR1Q21j4r-uazL4?usp=sharing

## 参考

こちらのノートブックを作成するにあたり、下記を参考にさせていただいております。

* [alfredplpl/RetrievalQA.py](https://gist.github.com/alfredplpl/57a6338bce8a00de9c9d95bbf1a6d06d)
* [Langchain Docs](https://python.langchain.com/docs/get_started/introduction)
* [Wikipedia「ONE_PIECE」](https://ja.m.wikipedia.org/wiki/ONE_PIECE)

